# Canhotos e as cobranças de penalty

## Introdução

Neste projeto rápido eu testo dois mitos frequentes sobre canhotos e cobranças de penalty. O primeiro mito é o de que "canhoto não sabe bater penalty". O segundo é o de que "canhoto perde penalty se bater no lado direito" (ponto de vista do batedor).

Eu testo esses dois mitos utilizando uma base com dados de todas as cobranças de penalty nas copas, entre 1982 e 2018. O conjunto de dados está disponível no link: https://www.kaggle.com/datasets/pablollanderos33/world-cup-penalty-shootouts?resource=download. 

## Importação dos dados e manipulações preliminares

In [1]:
# importando as bibliotecas básicas
import pandas as pd
import numpy as np

In [2]:
# importando os dados
df = pd.read_csv('WorldCupShootouts.csv')

In [3]:
# visualizando alguns registros do dataframe
df.head()

,Game_id,Team,Zone,Foot,Keeper,OnTarget,Goal,Penalty_Number,Elimination
0,1,FRA,7.0,R,R,1.0,1.0,1,0.0
1,1,GER,9.0,R,C,1.0,1.0,2,0.0
2,1,FRA,6.0,R,L,1.0,1.0,3,0.0
3,1,GER,2.0,R,C,1.0,1.0,4,0.0
4,1,FRA,9.0,R,L,1.0,1.0,5,0.0


In [4]:
# verificando a porcentagem de acertos das cobranças à esquerda, direita e no centro do gol

# dataframe contendo somente as cobranças à esquerda do gol
df_left = df[(df['Zone'] == 1) | (df['Zone'] == 4) | (df['Zone'] == 7)]

# dataframe contendo somente as cobranças no centro do gol
df_center = df[(df['Zone'] == 2) | (df['Zone'] == 5) | (df['Zone'] == 8)]

# dataframe contendo somente as cobranças à direita do gol
df_right = df[(df['Zone'] == 3) | (df['Zone'] == 6) | (df['Zone'] == 9)]

# porcentagem de acertos das cobranças à esquerda
l_success_rate = df_left['Goal'].sum()/len(df_left['Goal'])

# porcentagem de acertos das cobranças no centro 
c_success_rate = df_center['Goal'].sum()/len(df_center['Goal'])

# porcentagem de acertos das cobranças à direita
r_success_rate = df_right['Goal'].sum()/len(df_right['Goal'])


print('A porcentagem de acertos das cobranças à esquerda do gol é de: {0:.2%}'.format(l_success_rate))
print('A porcentagem de acertos das cobranças no centro do gol é de: {0:.2%}'.format(c_success_rate))
print('A porcentagem de acertos das cobranças à direita do gol é de: {0:.2%}'.format(r_success_rate))

A porcentagem de acertos das cobranças à esquerda do gol é de: 69.29%
A porcentagem de acertos das cobranças no centro do gol é de: 59.65%
A porcentagem de acertos das cobranças à direita do gol é de: 76.84%


In [5]:
# verificando as porcentagems de acerto dos canhotos nos dois lados do gol e no centro

# cria um novo dataframe somente com os dados dos canhotos
l_foot = df[df['Foot'] == 'L']

# cria um novo dataframe somente com as cobranças à esquerda do gol
l_foot_left = l_foot[(l_foot['Zone'] == 1) | (l_foot['Zone'] == 4) | (l_foot['Zone'] == 7)]
# cria um novo dataframe somente com as cobranças no centro do gol
l_foot_center = l_foot[(l_foot['Zone'] == 2) | (l_foot['Zone'] == 5) | (l_foot['Zone'] == 8)]
# cria um novo dataframe somente com as cobranças à direita do gol
l_foot_right = l_foot[(l_foot['Zone'] == 3) | (l_foot['Zone'] == 6) | (l_foot['Zone'] == 9)]

# calcula a porcentagem de acerto das cobranças à esquerda do gol
l_foot_l_success_rate = l_foot_left['Goal'].sum()/len(l_foot_left['Goal'])
# calcula a porcentagem de acerto das cobranças no centro do gol
l_foot_c_success_rate = l_foot_center['Goal'].sum()/len(l_foot_center['Goal'])
# calcula a porcentagem de acerto das cobranças à direita do gol
l_foot_r_success_rate = l_foot_right['Goal'].sum()/len(l_foot_right['Goal'])

print('A porcentagem de acerto dos canhotos nas cobranças à esquerda do gol é de: {:.2%}'.format(l_foot_l_success_rate))
print('A porcentagem de acerto dos canhotos nas cobranças no centro do gol é de: {:.2%}'.format(l_foot_c_success_rate))
print('A porcentagem de acerto dos canhotos nas cobranças à direita do gol é de: {:.2%}'.format(l_foot_r_success_rate))

A porcentagem de acerto dos canhotos nas cobranças à esquerda do gol é de: 66.67%
A porcentagem de acerto dos canhotos nas cobranças no centro do gol é de: 61.54%
A porcentagem de acerto dos canhotos nas cobranças à direita do gol é de: 72.73%


## Análise da diferença de acertos entre destros e canhotos

In [6]:
# dataframe com os dados dos destros apenas
r_foot = df[df['Foot'] == 'R']

# calcula a taxa de acertos dos destros
r_foot_success = r_foot['Goal'].sum()/len(r_foot)
# calcula a taxa de acertos dos canhotos
l_foot_success = l_foot['Goal'].sum()/len(l_foot)

print('A porcentagem de successo dos destros nas cobranças é de: {:.2%}'.format(r_foot_success))
print('A porcentagem de successo dos canhotos nas cobranças é de: {:.2%}'.format(l_foot_success))

A porcentagem de successo dos destros nas cobranças é de: 70.40%
A porcentagem de successo dos canhotos nas cobranças é de: 67.86%


Deseja-se agora testar se a diferença no sucesso nas cobranças é estatisticamente significante.

Esta questão será resolvida aplicando-se um teste do $\chi^2$. 

Tem-se aqui as seguintes hipóteses nula e alternativa:

   $$H_0 \text{: A taxa de acerto de penaltis de canhotos e destros não apresenta diferença significativa.}$$
   $$H_1 \text{: A taxa de acerto de penaltis dos destros é maior dos que a dos canhotos.}$$

In [7]:
# cria uma nova coluna que indica gols e perdas com "Yes" ou "No"
df['Goal2'] = df['Goal'].replace({0 : 'No', 1 : 'Yes'})

In [8]:
# matriz de contingência
cross_tab = pd.crosstab(df['Foot'], df['Goal2'], colnames = ['Goal'], margins = False)
cross_tab

Goal,No,Yes
Foot,,
L,18,38
R,66,157


In [9]:
# inicia a matriz esperada com zeros
E = np.zeros((2, 2))

In [10]:
# atribui os elementos da matriz esperada
for i in range(2):
    for j in range(2):
        E[i, j] = (cross_tab.iloc[i, :].sum())*(cross_tab.iloc[:, j].sum()) / 279.
        
# Matriz esperada
E

array([[ 16.86021505,  39.13978495],
       [ 67.13978495, 155.86021505]])

In [11]:
# calcula o valor de qui quadrado do teste
chi2_calc = 0

for i in range(2):
    for j in range(2):
        chi2_calc += (cross_tab.iloc[i, j] - E[i, j])**2 / E[i, j]
        
chi2_calc

0.1379277513005892

Para uma significância de 95%, $\chi^2_{\text{critico}} = 3.84.$

Também pode-se calcular o valor-p para o teste

In [12]:
from scipy import stats

# p-value do teste
p_value = 1 - stats.chi2.cdf(chi2_calc, 1)
p_value

0.7103498607364915

Como o p-valor é maior que o nível de significância a hipótese nula não pode ser rejeitada. Portanto, não há diferença significativa entre destros e canhotos nos acertos de penaltis.

## Testando o mito de que canhotos erram penaltis cobrados à direita do gol

In [13]:
# indica se a cobrança foi à esquerda, centro ou direita do gol
def shoot_side(zone):
    '''
    Recebe a zona do gol e retorna o lado da cobrança 
    
    :param zone: integer
    
    '''

    if zone in (1, 4, 7):
        return "Left"
    elif zone in (2, 5, 8):
        return "Center"
    else:
        return "Right"

In [14]:
# cria uma nova coluna com o lado do gol em que a cobrança foi efetuada
l_foot['shoot_side'] = l_foot['Zone'].apply(shoot_side)

/tmp/ipykernel_6767/1285718338.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  l_foot['shoot_side'] = l_foot['Zone'].apply(shoot_side)


Como o interesse nesta análise é comparar as cobranças feitas do lado direito e esquerdo, as cobranças realizadas no centro do gol são eliminadas da análise.

In [17]:
# cria um novo dataframe sem as cobranças no centro do gol 
l_foot = l_foot[l_foot['shoot_side'] != "Center"]
l_foot.head()

,Game_id,Team,Zone,Foot,Keeper,OnTarget,Goal,Penalty_Number,Elimination,shoot_side
17,2,FRA,9.0,L,R,1.0,1.0,6,0.0,Right
18,2,BRA,4.0,L,R,1.0,1.0,7,0.0,Left
22,3,GER,1.0,L,R,1.0,1.0,1,0.0,Left
23,3,MEX,6.0,L,L,1.0,1.0,2,0.0,Right
42,5,ROM,3.0,L,R,1.0,1.0,1,0.0,Right


A hipótese a ser testada aqui é a de que canhotos erram mais cobranças à direita do que à esquerda do gol.

Portanto, as hipóteses nula e alternativa podem ser formalizadas por:

   $$H_0 \text{: A taxa de acerto de penaltis dos canhotos das cobranças à direita e à esquerda do gol não apresenta diferença estatisticamente significativa.}$$
   $$H_1 \text{: A taxa de acertos de penaltis cobrados por canhotos à esquerda do gol é maior do que a taxa de acertos à direita do gol.}$$


In [25]:
# matriz de contingência
cross_tab2 = pd.crosstab(l_foot['shoot_side'], df['Goal2'], rownames = ['Goal Side'], colnames = ['Goal'], margins = False)
cross_tab2

Goal,No,Yes
Goal Side,,
Left,7,14
Right,6,16


In [26]:
# inicia a matriz esperada com zeros
E2 = np.zeros((2, 2))

In [29]:
# atribui os elementos da matriz esperada
for i in range(2):
    for j in range(2):
        E2[i, j] = (cross_tab2.iloc[i, :].sum())*(cross_tab2.iloc[:, j].sum()) / 43.
E2

array([[ 6.34883721, 14.65116279],
       [ 6.65116279, 15.34883721]])

In [30]:
# calcula o valor de qui quadrado 
chi2_calc2 = 0

for i in range(2):
    for j in range(2):
        chi2_calc2 += (cross_tab2.iloc[i, j] - E2[i, j])**2 / E2[i, j]
        
chi2_calc2

0.1871017871017871

Considerando novamente um nível de significância de 95%, o valor de $\chi^2_{\text{calc}}$ é menor que o valor crítico $\chi^2_{\text{critico}} = 3.84.$

Por fim, o valor-p é calculado.

In [32]:
# p-value do teste
p_value2 = 1 - stats.chi2.cdf(chi2_calc2, 1)
p_value2

0.6653398015268104

Mais uma vez, tem-se um valor-p maior do que o nível de significância e a hipótese nula não pode ser descartada. 

Conclui-se, portanto, que não há diferença significativa nos acertos das cobranças à esquerda e à direita do gol pelos canhotos.  